# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [84]:
import os
import pandas as pd
import numpy as np
import re
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans
import scipy.stats as stats
PUIdata = os.getenv('PUIDATA')

In [8]:
PUIdata

'/home/cusp/gtp232/PUIdata'

In [9]:
# Load dataset
data_311 = pd.read_csv(PUIdata + '/erm2-nwe9.csv')
data_311[['Community Board']].head()

,Community Board
0,10 BROOKLYN
1,12 BRONX
2,01 STATEN ISLAND
3,10 BROOKLYN
4,10 BROOKLYN


In [10]:
#Load full dataset
'''
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9').loc[:,\
                                                                ['Complaint Type','Community Board', 'Borough']]
                                                                
                                                            '''

"\ndata = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9').loc[:,                                                                ['Complaint Type','Community Board', 'Borough']]\n                                                                \n                                                            "

# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [11]:
data_311.columns

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',

In [12]:
data_311.Borough.unique()

array(['BROOKLYN', 'BRONX', 'STATEN ISLAND', 'QUEENS', 'MANHATTAN',
       'Unspecified'], dtype=object)

In [13]:
# Check if all Boroughs and Community Districts are represented in the Data 
community_board_unique = data_311['Community Board'].unique()
community_board_unique

array(['10 BROOKLYN', '12 BRONX', '01 STATEN ISLAND', '14 QUEENS',
       '03 MANHATTAN', '04 BRONX', 'Unspecified MANHATTAN', '02 QUEENS',
       '01 MANHATTAN', '03 STATEN ISLAND', '11 QUEENS', '02 STATEN ISLAND',
       '06 BROOKLYN', '12 MANHATTAN', '10 QUEENS', '09 QUEENS', '11 BRONX',
       '03 BROOKLYN', '02 BROOKLYN', '15 BROOKLYN', '04 MANHATTAN',
       'Unspecified BROOKLYN', '08 BRONX', '0 Unspecified', '04 QUEENS',
       '14 BROOKLYN', '01 BRONX', '09 BRONX', '05 QUEENS', '08 MANHATTAN',
       '18 BROOKLYN', '12 BROOKLYN', '04 BROOKLYN', '06 MANHATTAN',
       '03 BRONX', '11 BROOKLYN', '02 MANHATTAN', '05 MANHATTAN',
       '12 QUEENS', '01 BROOKLYN', '13 QUEENS', '05 BRONX', '10 MANHATTAN',
       '07 QUEENS', '16 BROOKLYN', '10 BRONX', '09 BROOKLYN', '03 QUEENS',
       '11 MANHATTAN', '07 MANHATTAN', '13 BROOKLYN', '05 BROOKLYN',
       '01 QUEENS', '07 BRONX', '09 MANHATTAN', 'Unspecified BRONX',
       '08 QUEENS', '06 BRONX', '08 BROOKLYN', '17 BROOKLYN',
       

In [14]:
# How many unique values do we have? 
len(community_board_unique)

65

In [15]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.
#data_311 = data_311[["Created Date", "Agency", "Complaint Type", "Community Board", "Latitude", "Longitude"]]
data_311 = data_311[~data_311['Community Board'].str.contains("Unspecified")]
data_311.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,34666003,10/31/2016 11:16:02 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11228.0,930 73 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.626284,-74.014215,"(40.626284457572716, -74.01421534164334)"
1,34666002,10/31/2016 11:15:18 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,10470.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.898450,-73.854298,"(40.89844965691687, -73.85429832975592)"
2,34668139,10/31/2016 11:03:52 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,10314.0,55 PERRY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.611195,-74.130085,"(40.611194968031164, -74.13008460276865)"
3,34665569,10/31/2016 10:53:16 AM,10/31/2016 10:53:16 AM,DOT,Department of Transportation,Street Condition,Pothole,NaN,11209.0,87 70 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.637114,-74.032936,"(40.63711429954738, -74.0329355116319)"
4,34666688,10/31/2016 10:50:59 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11209.0,87 70 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.637114,-74.032936,"(40.63711429954738, -74.0329355116319)"


In [16]:
# Check for duplicates? Are these plausible?
community_board_unique = data_311['Community Board'].unique()
len(community_board_unique)

60

In [17]:
sorted(community_board_unique)

['01 BRONX',
 '01 BROOKLYN',
 '01 MANHATTAN',
 '01 QUEENS',
 '01 STATEN ISLAND',
 '02 BRONX',
 '02 BROOKLYN',
 '02 MANHATTAN',
 '02 QUEENS',
 '02 STATEN ISLAND',
 '03 BRONX',
 '03 BROOKLYN',
 '03 MANHATTAN',
 '03 QUEENS',
 '03 STATEN ISLAND',
 '04 BRONX',
 '04 BROOKLYN',
 '04 MANHATTAN',
 '04 QUEENS',
 '05 BRONX',
 '05 BROOKLYN',
 '05 MANHATTAN',
 '05 QUEENS',
 '06 BRONX',
 '06 BROOKLYN',
 '06 MANHATTAN',
 '06 QUEENS',
 '07 BRONX',
 '07 BROOKLYN',
 '07 MANHATTAN',
 '07 QUEENS',
 '08 BRONX',
 '08 BROOKLYN',
 '08 MANHATTAN',
 '08 QUEENS',
 '09 BRONX',
 '09 BROOKLYN',
 '09 MANHATTAN',
 '09 QUEENS',
 '10 BRONX',
 '10 BROOKLYN',
 '10 MANHATTAN',
 '10 QUEENS',
 '11 BRONX',
 '11 BROOKLYN',
 '11 MANHATTAN',
 '11 QUEENS',
 '12 BRONX',
 '12 BROOKLYN',
 '12 MANHATTAN',
 '12 QUEENS',
 '13 BROOKLYN',
 '13 QUEENS',
 '14 BROOKLYN',
 '14 QUEENS',
 '15 BROOKLYN',
 '16 BROOKLYN',
 '17 BROOKLYN',
 '18 BROOKLYN',
 '81 QUEENS']

In [18]:
# What about missing values? Can you detect any patterns? 
for i in data_311.columns:
    print i
    print pd.isnull(data_311[i]).sum()

Unique Key
0
Created Date
0
Closed Date
489
Agency
0
Agency Name
0
Complaint Type
0
Descriptor
11
Location Type
109
Incident Zip
0
Incident Address
110
Street Name
110
Cross Street 1
256
Cross Street 2
256
Intersection Street 1
848
Intersection Street 2
848
Address Type
9
City
0
Landmark
948
Facility Type
361
Status
0
Due Date
247
Resolution Description
68
Resolution Action Updated Date
205
Community Board
0
Borough
0
X Coordinate (State Plane)
4
Y Coordinate (State Plane)
4
Park Facility Name
0
Park Borough
0
School Name
0
School Number
0
School Region
2
School Code
2
School Phone Number
0
School Address
0
School City
0
School State
0
School Zip
0
School Not Found
251
School or Citywide Complaint
948
Vehicle Type
948
Taxi Company Borough
946
Taxi Pick Up Location
942
Bridge Highway Name
941
Bridge Highway Direction
941
Road Ramp
941
Bridge Highway Segment
941
Garage Lot Name
948
Ferry Direction
948
Ferry Terminal Name
948
Latitude
4
Longitude
4
Location
4


In [19]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?
data_311_res = data_311[["Community Board", "Unique Key", "Created Date", "Closed Date", "Agency", "Agency Name", "Complaint Type", "Descriptor", \
                        "Incident Zip", "Incident Address", "Latitude", "Longitude", "Status", "Resolution Description"]]
data_311_res[~pd.isnull(data_311_res["Resolution Description"])]

,Community Board,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Incident Zip,Incident Address,Latitude,Longitude,Status,Resolution Description
0,10 BROOKLYN,34666003,10/31/2016 11:16:02 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11228.0,930 73 STREET,40.626284,-74.014215,Open,The Department of Transportation referred this...
1,12 BRONX,34666002,10/31/2016 11:15:18 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,10470.0,NaN,40.898450,-73.854298,Open,The Department of Transportation referred this...
2,01 STATEN ISLAND,34668139,10/31/2016 11:03:52 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,10314.0,55 PERRY AVENUE,40.611195,-74.130085,Open,The Department of Transportation referred this...
3,10 BROOKLYN,34665569,10/31/2016 10:53:16 AM,10/31/2016 10:53:16 AM,DOT,Department of Transportation,Street Condition,Pothole,11209.0,87 70 STREET,40.637114,-74.032936,Closed,The Department of Transportation determined th...
4,10 BROOKLYN,34666688,10/31/2016 10:50:59 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11209.0,87 70 STREET,40.637114,-74.032936,Open,The Department of Transportation referred this...
5,14 QUEENS,34663951,10/31/2016 10:48:22 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11692.0,560 BEACH 66 STREET,40.596188,-73.795472,Open,The Department of Transportation referred this...
6,03 MANHATTAN,34666482,10/31/2016 10:46:06 AM,NaN,DSNY,Department of Sanitation,Graffiti,Graffiti,10009.0,549 EAST 13 STREET,40.729251,-73.979708,Open,The graffiti on this property has been schedul...
7,04 BRONX,34665276,10/31/2016 10:21:33 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,10451.0,NaN,40.825724,-73.927153,Open,The Department of Transportation referred this...
8,01 STATEN ISLAND,34668151,10/31/2016 10:13:00 AM,NaN,DSNY,Department of Sanitation,Electronics Waste,Recycling Electronics,10302.0,113 BURDEN AVENUE,40.631742,-74.140065,Open,The Department of Sanitation is in the process...
10,01 STATEN ISLAND,34663275,10/31/2016 09:57:00 AM,NaN,DSNY,Department of Sanitation,Electronics Waste,Recycling Electronics,10310.0,391 OAKLAND AVENUE,40.632262,-74.110671,Open,The Department of Sanitation is in the process...


In [20]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 

data_311_res['bad_community_district'] = [int(i[:2])>18 for i in data_311_res['Community Board']]
data_311_res.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Community Board,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Incident Zip,Incident Address,Latitude,Longitude,Status,Resolution Description,bad_community_district
0,10 BROOKLYN,34666003,10/31/2016 11:16:02 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11228.0,930 73 STREET,40.626284,-74.014215,Open,The Department of Transportation referred this...,False
1,12 BRONX,34666002,10/31/2016 11:15:18 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,10470.0,NaN,40.898450,-73.854298,Open,The Department of Transportation referred this...,False
2,01 STATEN ISLAND,34668139,10/31/2016 11:03:52 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,10314.0,55 PERRY AVENUE,40.611195,-74.130085,Open,The Department of Transportation referred this...,False
3,10 BROOKLYN,34665569,10/31/2016 10:53:16 AM,10/31/2016 10:53:16 AM,DOT,Department of Transportation,Street Condition,Pothole,11209.0,87 70 STREET,40.637114,-74.032936,Closed,The Department of Transportation determined th...,False
4,10 BROOKLYN,34666688,10/31/2016 10:50:59 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11209.0,87 70 STREET,40.637114,-74.032936,Open,The Department of Transportation referred this...,False


In [21]:
# Drop the marked districts
data_311_res2 = data_311_res[data_311_res["bad_community_district"] == False]
data_311_res2.head(30)

,Community Board,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Incident Zip,Incident Address,Latitude,Longitude,Status,Resolution Description,bad_community_district
0,10 BROOKLYN,34666003,10/31/2016 11:16:02 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11228.0,930 73 STREET,40.626284,-74.014215,Open,The Department of Transportation referred this...,False
1,12 BRONX,34666002,10/31/2016 11:15:18 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,10470.0,NaN,40.898450,-73.854298,Open,The Department of Transportation referred this...,False
2,01 STATEN ISLAND,34668139,10/31/2016 11:03:52 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,10314.0,55 PERRY AVENUE,40.611195,-74.130085,Open,The Department of Transportation referred this...,False
3,10 BROOKLYN,34665569,10/31/2016 10:53:16 AM,10/31/2016 10:53:16 AM,DOT,Department of Transportation,Street Condition,Pothole,11209.0,87 70 STREET,40.637114,-74.032936,Closed,The Department of Transportation determined th...,False
4,10 BROOKLYN,34666688,10/31/2016 10:50:59 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11209.0,87 70 STREET,40.637114,-74.032936,Open,The Department of Transportation referred this...,False
5,14 QUEENS,34663951,10/31/2016 10:48:22 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11692.0,560 BEACH 66 STREET,40.596188,-73.795472,Open,The Department of Transportation referred this...,False
6,03 MANHATTAN,34666482,10/31/2016 10:46:06 AM,NaN,DSNY,Department of Sanitation,Graffiti,Graffiti,10009.0,549 EAST 13 STREET,40.729251,-73.979708,Open,The graffiti on this property has been schedul...,False
7,04 BRONX,34665276,10/31/2016 10:21:33 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,10451.0,NaN,40.825724,-73.927153,Open,The Department of Transportation referred this...,False
8,01 STATEN ISLAND,34668151,10/31/2016 10:13:00 AM,NaN,DSNY,Department of Sanitation,Electronics Waste,Recycling Electronics,10302.0,113 BURDEN AVENUE,40.631742,-74.140065,Open,The Department of Sanitation is in the process...,False
10,01 STATEN ISLAND,34663275,10/31/2016 09:57:00 AM,NaN,DSNY,Department of Sanitation,Electronics Waste,Recycling Electronics,10310.0,391 OAKLAND AVENUE,40.632262,-74.110671,Open,The Department of Sanitation is in the process...,False


In [22]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level

df_sample = data_311_res2.groupby('Community Board')[['Complaint Type']].count()
#df_sample = pd.DataFrame(df_sample)
#df_sample = pd.DataFrame(df_sample)
df_sample.reset_index()
df_sample.sort(['Complaint Type'], ascending=False)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Complaint Type
Community Board,
12 MANHATTAN,48
14 BROOKLYN,36
09 QUEENS,36
11 BROOKLYN,33
07 MANHATTAN,30
01 BROOKLYN,29
03 BROOKLYN,28
03 QUEENS,24
01 QUEENS,23


In [23]:
# Save reduced data frame (Community District level)
df_311_final = df_sample

# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

In [24]:
# Population by Community District
df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")

In [25]:
# Check variables in file
df_pop.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [26]:
# How many community districts are in file? 
df_pop.cd_id.count()

59

In [27]:
df_pop.columns[10:150]

Index([u'% Population 5 Years And Over: Spanish or Spanish Creole: Speak English "very Well"',
       u'% Population 5 Years And Over: Spanish or Spanish Creole: Speak English Less Than "very Well"',
       u'Population 25 Years and over:',
       u'Population 25 Years and over: Less Than High School',
       u'Population 25 Years and over: High School Graduate (includes equivalency)',
       u'Population 25 Years and over: Some college',
       u'Population 25 Years and over: Bachelor's degree',
       u'Population 25 Years and over: Master's degree',
       u'Population 25 Years and over: Professional school degree',
       u'Population 25 Years and over: Doctorate degree',
       ...
       u'Per capita income (In 2014 Inflation Adjusted Dollars)',
       u'Employed Civilian Population 16 Years And Over:.1',
       u'Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       u'Employed Civilian Population 16 Years And Over: Service occ

In [28]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311
df_pop_small = df_pop[["cd_id", "Total Population", "% Total Population: 18 to 24 Years", "% Total Population: 25 to 34 Years", \
       "% Total Population: 35 to 44 Years", "Per capita income (In 2014 Inflation Adjusted Dollars)", \
       "Employed Civilian Population 16 Years And Over:.1"]]
df_pop_small['pct_employed'] = df_pop_small['Employed Civilian Population 16 Years And Over:.1'] / df_pop_small['Total Population']
df_pop_small.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,cd_id,Total Population,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,Per capita income (In 2014 Inflation Adjusted Dollars),Employed Civilian Population 16 Years And Over:.1,pct_employed
0,BX08,106737,10.73,15.04,11.32,30730,47433,0.444391
1,BX12,134644,11.35,14.29,12.57,21523,59430,0.441386
2,BX10,121209,8.62,13.74,12.78,28586,53487,0.441279
3,BX11,135839,8.10,17.43,14.09,22714,60765,0.447331
4,BX03,172247,14.24,14.89,12.38,12662,60127,0.349074


In [29]:
# Save data frame

In [30]:
# Infrastructure by Community District
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [31]:
# Check variables in file
df_infr.head(100)

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980
5,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980
6,3603706,79500US3603706,"NYC-Bronx Community District 7--Bedford Park, ...",47252,31468,0,1598,502,1096,18280,...,1624,4655,11129,66.60,47252,38899,0,31021,7878,8353
7,3603707,79500US3603707,NYC-Bronx Community District 5--Morris Heights...,44699,26332,54,956,352,604,11333,...,1062,2980,15387,58.91,44699,34397,54,25772,8571,10302
8,3603708,79500US3603708,"NYC-Bronx Community District 4--Concourse, New...",47935,29376,316,301,73,228,14610,...,2374,3701,14858,61.28,47935,34692,316,27971,6405,13243
9,3603709,79500US3603709,"NYC-Bronx Community District 9--Castle Hill, N...",64011,45976,180,1536,296,1240,23999,...,1663,1871,16164,71.83,64011,53072,180,45261,7631,10939


In [32]:
len(df_infr.columns)

31

In [33]:
df_infr.columns

Index([u'FIPS', u'Geographic Identifier', u'Qualifying Name', u'Households',
       u'Households: With An Internet Subscription',
       u'Households: Dial-Up Alone', u'Households: Dsl',
       u'Households: With Mobile Broadband',
       u'Households: Without Mobile Broadband', u'Households: Cable Modem',
       u'Households: With Mobile Broadband.1',
       u'Households: Without Mobile Broadband.1', u'Households: Fiber-Optic',
       u'Households: With Mobile Broadband.2',
       u'Households: Without Mobile Broadband.2',
       u'Households: Satellite Internet Service',
       u'Households: With Mobile Broadband.3',
       u'Households: Without Mobile Broadband.3',
       u'Households: Two or More Fixed Broadband Types, or Other',
       u'Households: With Mobile Broadband.4',
       u'Households: Without Mobile Broadband.4',
       u'Households: Mobile Broadband Alone or With Dialup',
       u'Households: Internet Access Without A Subscription',
       u'Households: No Internet Acc

In [34]:
# How many community districts are in file? 
len(df_infr['Qualifying Name'].unique())

55

In [35]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data
df_infr_small = df_infr[["FIPS", "Geographic Identifier", "Qualifying Name", "Households", "Households: With An Internet Subscription",\
                        "Households: No Internet Access", "Households: Has A Computer", "Households: Without An Internet Subscription",\
                        "Households: With Dial-Up Internet Subscription Alone", "Households: With A Broadband Internet Subscription"]]
df_infr_small.groupby('Qualifying Name')['Households'].sum(),

(Qualifying Name
 NYC-Bronx Community District 1 & 2--Hunts Point, New York                                   104382
 NYC-Bronx Community District 10--Co-op City, New York                                        47050
 NYC-Bronx Community District 11--Pelham Parkway, New York                                    44922
 NYC-Bronx Community District 12--Wakefield, New York                                         44830
 NYC-Bronx Community District 3 & 6--Belmont, New York                                       115112
 NYC-Bronx Community District 4--Concourse, New York                                          47935
 NYC-Bronx Community District 5--Morris Heights, New York                                     44699
 NYC-Bronx Community District 7--Bedford Park, New York                                       47252
 NYC-Bronx Community District 8--Riverdale, New York                                          42035
 NYC-Bronx Community District 9--Castle Hill, New York                             

In [36]:
# Aggregate internet type by high and low connections
df_infr_small['low_connect'] = df_infr_small['Households: With Dial-Up Internet Subscription Alone']\
+ df_infr_small['Households: Without An Internet Subscription']
df_infr_small['hi_connect'] = df_infr_small['Households: With A Broadband Internet Subscription']
df_infr_final = df_infr_small[["Qualifying Name", "Households", "low_connect", "hi_connect"]]
df_infr_final.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Qualifying Name,Households,low_connect,hi_connect
0,"NYC-Bronx Community District 8--Riverdale, New...",42035,4105,30943
1,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,5265,31435
2,"NYC-Bronx Community District 10--Co-op City, N...",47050,6367,32333
3,NYC-Bronx Community District 11--Pelham Parkwa...,44922,5959,31278
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,9168,33408


In [37]:
# Save data frame 

# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [38]:
df_311_final.head()

,Complaint Type
Community Board,
01 BRONX,8
01 BROOKLYN,29
01 MANHATTAN,13
01 QUEENS,23
01 STATEN ISLAND,16


In [39]:
# Harmonize identifier of dataframe 1

dict_boro = {'BRONX': 'BX', 'BROOKLYN': 'BK', 'MANHATTAN': 'MN', 'STATEN ISLAND': 'SI', 'QUEENS': 'QN'}

df_311_final['cd_id'] = '0'

for i in range(len(df_311_final)):
    df_311_final['cd_id'][i] = dict_boro[df_311_final.index[i][3:]] + df_311_final.index[i][:2]

df_311_final.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Complaint Type,cd_id
Community Board,,
01 BRONX,8,BX01
01 BROOKLYN,29,BK01
01 MANHATTAN,13,MN01
01 QUEENS,23,QN01
01 STATEN ISLAND,16,SI01


In [40]:
# Harmonize identifier of dataframe 2

df_pop_small.head()

,cd_id,Total Population,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,Per capita income (In 2014 Inflation Adjusted Dollars),Employed Civilian Population 16 Years And Over:.1,pct_employed
0,BX08,106737,10.73,15.04,11.32,30730,47433,0.444391
1,BX12,134644,11.35,14.29,12.57,21523,59430,0.441386
2,BX10,121209,8.62,13.74,12.78,28586,53487,0.441279
3,BX11,135839,8.10,17.43,14.09,22714,60765,0.447331
4,BX03,172247,14.24,14.89,12.38,12662,60127,0.349074


In [41]:
m = re.search("[0-9]+", df_infr_small['Qualifying Name'][4]).group(0)
len(m)

1

In [42]:
# Harmonize identifier of dataframe 3

dict_boro2 = {'Bronx': 'BX', 'Brooklyn': 'BK', 'Manhattan': 'MN', 'Staten': 'SI', 'Queens': 'QN'}

def add_zero(f):
    if len(f) == 1:
        return '0'+f
    else:
        return f

df_infr_final['cd_num'] = [re.search("[0-9]+", i).group(0) for i in df_infr_final['Qualifying Name']]
df_infr_final['cd_num_2'] = [add_zero(i) for i in df_infr_final['cd_num']]

df_infr_final['boro'] = [dict_boro2[re.search('[^\s]+', i[4:]).group(0)] for i in df_infr_final['Qualifying Name']]

df_infr_final['cd_id'] = df_infr_final['boro'] + df_infr_final['cd_num_2']


df_infr_final_2 = df_infr_final[["cd_id", "Households", "low_connect", "hi_connect"]]


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,cd_id,Households,low_connect,hi_connect
0,BX08,42035,4105,30943
1,BX12,44830,5265,31435
2,BX10,47050,6367,32333
3,BX11,44922,5959,31278
4,BX03,57556,9168,33408


In [53]:
#remove duplicates
df_infr_final_2 = df_infr_final_2.drop_duplicates()
df_infr_final_2.head(20)

,cd_id,Households,low_connect,hi_connect
0,BX08,42035,4105,30943
1,BX12,44830,5265,31435
2,BX10,47050,6367,32333
3,BX11,44922,5959,31278
4,BX03,57556,9168,33408
6,BX07,47252,7878,31021
7,BX05,44699,8625,25772
8,BX04,47935,6721,27971
9,BX09,64011,7811,45261
10,BX01,52191,8183,30958


In [54]:
# Link the 3 dataframes
df_311_pop = df_311_final.merge(df_pop_small, left_on='cd_id', right_on='cd_id', how='left')
df_311_pop_infr = df_311_pop.merge(df_infr_final_2, left_on='cd_id', right_on='cd_id', how='left')
df_311_pop_infr.head(20)

,Complaint Type,cd_id,Total Population,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,Per capita income (In 2014 Inflation Adjusted Dollars),Employed Civilian Population 16 Years And Over:.1,pct_employed,Households,low_connect,hi_connect
0,8,BX01,167147.0,12.76,18.26,12.04,11370.0,51760.0,0.309668,52191.0,8183.0,30958.0
1,29,BK01,154713.0,8.52,29.88,13.70,34897.0,85020.0,0.549534,62990.0,6512.0,46148.0
2,13,MN01,159903.0,8.55,28.38,18.14,105300.0,100992.0,0.631583,83976.0,5551.0,74339.0
3,23,QN01,182860.0,9.26,27.12,15.74,31664.0,103065.0,0.563628,75758.0,5290.0,60733.0
4,16,SI01,176338.0,10.93,15.08,12.94,26988.0,75432.0,0.427769,62047.0,3797.0,46362.0
5,3,BX02,167147.0,12.76,18.26,12.04,11370.0,51760.0,0.309668,NaN,NaN,NaN
6,10,BK02,139070.0,9.57,22.68,17.05,52259.0,72363.0,0.520335,57188.0,4756.0,45378.0
7,11,MN02,159903.0,8.55,28.38,18.14,105300.0,100992.0,0.631583,NaN,NaN,NaN
8,9,QN02,150100.0,7.88,21.98,17.65,28789.0,82022.0,0.546449,52641.0,3152.0,43902.0
9,10,SI02,126200.0,7.82,12.51,12.48,33525.0,57094.0,0.452409,45266.0,4157.0,34435.0


In [55]:
df_311_pop_infr.columns

Index([u'Complaint Type', u'cd_id', u'Total Population',
       u'% Total Population: 18 to 24 Years',
       u'% Total Population: 25 to 34 Years',
       u'% Total Population: 35 to 44 Years',
       u'Per capita income (In 2014 Inflation Adjusted Dollars)',
       u'Employed Civilian Population 16 Years And Over:.1', u'pct_employed',
       u'Households', u'low_connect', u'hi_connect'],
      dtype='object')

In [59]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?
'''
Geoff solution number 1: run linear regression and see if demo & infrastructure 
is a significant predictor of number of complaints. If yes, then there exists differences in community districts
'''
df_311_pop_infr.columns = ['complaint_type', 'cd_id', 'total_population', 'total_pop_18_24_pct',\
                          'total_pop_25_34_pct', 'total_pop_35_44_pct', 'income_per_cap', 'employed_pop',\
                          'pct_employed', 'households', 'low_connect', 'hi_connect']
results = smf.ols('complaint_type ~ total_population + total_pop_18_24_pct + total_pop_25_34_pct + total_pop_35_44_pct \
                    + income_per_cap + employed_pop + pct_employed + households + low_connect + hi_connect', data = df_311_pop_infr).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         complaint_type   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     1.863
Date:                Sun, 06 Nov 2016   Prob (F-statistic):             0.0779
Time:                        15:28:57   Log-Likelihood:                -183.97
No. Observations:                  54   AIC:                             389.9
Df Residuals:                      43   BIC:                             411.8
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
Intercept               3.7066     50.243      0.074      0.942       -97.618   105.031
total_population       -0.0001      0.000     -0.438      0.664        -0.001     0.001
total_pop_18_24_pct     0.0873      0.614      0.142      0.888        -1.152     1.327
total_pop_25_34_pct     0.0495      0.461      0.107      0.915        -0.879     0.978
total_pop_35_44_pct     0.0844      0.780      0.108      0.914        -1.489     1.657
income_per_cap         -0.0002      0.000     -1.293      0.203        -0.001     0.000
employed_pop            0.0004      0.001      0.538      0.594        -0.001     0.002
pct_employed           -3.9329    113.379     -0.035      0.972      -232.584   224.719
households           9.448e-05      0.001      0.183      0.855        -0.001     0.001
low_connect             0.0012      0.001      1.143      0.259        -0.001     0.003
hi_connect            1.54e-07      0.001      0.000      1.000        -0.001     0.001
==============================================================================
Omnibus:                       12.322   Durbin-Watson:                   2.628
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               12.847
Skew:                           1.009   Prob(JB):                      0.00162
Kurtosis:                       4.281   Cond. No.                     2.14e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.14e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [64]:
df_311_pop_infr.iloc[:,2:].head()

,total_population,total_pop_18_24_pct,total_pop_25_34_pct,total_pop_35_44_pct,income_per_cap,employed_pop,pct_employed,households,low_connect,hi_connect
0,167147.0,12.76,18.26,12.04,11370.0,51760.0,0.309668,52191.0,8183.0,30958.0
1,154713.0,8.52,29.88,13.70,34897.0,85020.0,0.549534,62990.0,6512.0,46148.0
2,159903.0,8.55,28.38,18.14,105300.0,100992.0,0.631583,83976.0,5551.0,74339.0
3,182860.0,9.26,27.12,15.74,31664.0,103065.0,0.563628,75758.0,5290.0,60733.0
4,176338.0,10.93,15.08,12.94,26988.0,75432.0,0.427769,62047.0,3797.0,46362.0


In [75]:
df_311_pop_infr_no_nans = df_311_pop_infr[np.isfinite(df_311_pop_infr['households'])]
df_311_pop_infr_no_nans = df_311_pop_infr_no_nans[np.isfinite(df_311_pop_infr_no_nans['low_connect'])]
df_311_pop_infr_no_nans = df_311_pop_infr_no_nans[np.isfinite(df_311_pop_infr_no_nans['hi_connect'])]
df_311_pop_infr_no_nans = df_311_pop_infr_no_nans[np.isfinite(df_311_pop_infr_no_nans['total_population'])]

In [76]:
df_311_pop_infr_no_nans.iloc[:,2:].head(60)

,total_population,total_pop_18_24_pct,total_pop_25_34_pct,total_pop_35_44_pct,income_per_cap,employed_pop,pct_employed,households,low_connect,hi_connect
0,167147.0,12.76,18.26,12.04,11370.0,51760.0,0.309668,52191.0,8183.0,30958.0
1,154713.0,8.52,29.88,13.70,34897.0,85020.0,0.549534,62990.0,6512.0,46148.0
2,159903.0,8.55,28.38,18.14,105300.0,100992.0,0.631583,83976.0,5551.0,74339.0
3,182860.0,9.26,27.12,15.74,31664.0,103065.0,0.563628,75758.0,5290.0,60733.0
4,176338.0,10.93,15.08,12.94,26988.0,75432.0,0.427769,62047.0,3797.0,46362.0
6,139070.0,9.57,22.68,17.05,52259.0,72363.0,0.520335,57188.0,4756.0,45378.0
8,150100.0,7.88,21.98,17.65,28789.0,82022.0,0.546449,52641.0,3152.0,43902.0
9,126200.0,7.82,12.51,12.48,33525.0,57094.0,0.452409,45266.0,4157.0,34435.0
10,172247.0,14.24,14.89,12.38,12662.0,60127.0,0.349074,57556.0,9168.0,33408.0
11,133235.0,12.75,19.43,11.55,21424.0,58739.0,0.440868,50688.0,6790.0,32660.0


In [77]:
'''
Geoff solution number 2: run k-means clustering on demo / infrastructure characteristics (k=2)

compare number of complaints between the two groups - if significant difference in number, then yes
'''


n=2 # number of clusters
dd = df_311_pop_infr_no_nans.iloc[:,2:] #data


#train the model.
km = KMeans(random_state=324, n_clusters=n)
res = km.fit(dd)
#result.
print(res.labels_)
#print res.predict(dd)
print(km.cluster_centers_)

[0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1]
[[  1.36637778e+05   9.95111111e+00   1.71061111e+01   1.36622222e+01
    2.62680556e+04   6.11562222e+04   4.48533523e-01   4.91826667e+04
    5.44822222e+03   3.45703611e+04]
 [  1.91244778e+05   9.01722222e+00   1.87561111e+01   1.43611111e+01
    4.52722222e+04   9.66746667e+04   5.11187803e-01   7.37650000e+04
    5.68461111e+03   5.89209444e+04]]


In [86]:
# use this data set:
#save your clustering result to cluster_label

df_311_pop_infr_no_nans['Cluster'] = res1.predict(dd) # Make sure here is your clustering result.

In [82]:
#separate groups
df_group_0 = df_311_pop_infr_no_nans[df_311_pop_infr_no_nans.Cluster == 0]
df_group_1 = df_311_pop_infr_no_nans[df_311_pop_infr_no_nans.Cluster == 1]

In [85]:
ks = stats.ks_2samp(df_group_0.complaint_type, df_group_1.complaint_type)
print (ks)

Ks_2sampResult(statistic=0.2777777777777779, pvalue=0.26530828160637782)


## Conclusion
I split the data into two groups, each group composed of the most similar community districts based on all the demographic and infrastructure attributes I kept. I then compared the distribution of complaints of each to one another to see if they were significantly different at the p=0.05 level. The null that the distribution of complaints is NOT significantly different is NOT rejected (the p-value is 0.265). Therefore, I've come to the conclusion that there is not a significant difference between 311 complaints across two groups that are different based on demographic and infrastructure attributes. 